In [ ]:
!pip install tensorflow

In [ ]:
!pip install pandas
!pip install scikit-learn


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 35.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Lambda, Layer, Input, Dense, Concatenate
from tensorflow.keras.layers import Flatten, Conv1D, Reshape, GlobalAveragePooling2D,LSTM, BatchNormalization
import json
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Set TensorFlow to use only the CPU
tf.config.set_visible_devices([], 'GPU')

In [12]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs detected:")
    for gpu in gpus:
        print(f"- {gpu}")
else:
    print("No GPUs detected.")

No GPUs detected.


In [13]:
def get_csrf_token(session):
    """Fetches the CSRF token from the login page."""
    login_url = "https://www.heartharena.com/login"
    response = session.get(login_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    csrf_token = soup.find("input", {"name": "_csrf_token"})["value"]
    return csrf_token

def login(username, password):
    """Logs in to the HearthArena website."""
    url = "https://www.heartharena.com/login_check"
    session = requests.Session()

    # Get CSRF token
    csrf_token = get_csrf_token(session)

    # Prepare payload
    payload = {
        "_username": username,
        "_password": password,
        "_csrf_token": csrf_token,
        "_remember_me": "on"  # Optional, based on whether you want to stay logged in
    }

    # Submit the login form
    response = session.post(url, data=payload)

    # Check if login was successful
    if response.status_code == 200:
        print("Login successful!")
    else:
        print("Login failed.")

    return session



In [14]:
def getRuns(arenaURL,session):
    # Initialize an empty list to store the links
    response = session.get(arenaURL)
    links = []
    end = BeautifulSoup(response.text, 'html')
    runs = end.find('tbody')
    # Find all 'a' tags within 'tr' elements and filter for those with 'href' attribute starting with '/arena-run/'
    for link in runs.find_all('tr'):
        anchor_tags = link.find_all('a')
        for tag in anchor_tags:
            href = tag.get('href')
            if href and href.startswith('/arena-run/'):
                links.append('https://www.heartharena.com'+href)
    links = list(set(links))
    return links

In [15]:
def getStatsFromRun(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    nameExtract = soup.select('h1.class')[0].text.strip()
    parts = nameExtract.split('/')

    wins = 0
    deckList = 0
    champ = ""
    hasData = len(soup.find_all('span', class_='wins'))
    print(hasData)
    if hasData == 1:
        champ = parts[0].strip()
        wins = int(soup.find_all('span', class_='wins')[0].text)
        arenaDeck = soup.find_all('ul', class_="deckList")[0]
        cardNames = arenaDeck.find_all('span', class_='name')
        cardAmount = arenaDeck.find_all('span', class_='quantity')
        # Extract and convert numeric values and names
        namesList = [(name.text.strip()) for name in cardNames]
        amountList = [int(amount.text.strip()) for amount in cardAmount]

        # Initialize an empty list to hold the repeated names
        deckList = []

        # Iterate through the names and amounts simultaneously
        for name, amount in zip(namesList, amountList):
            # Repeat the name 'amount' times and extend the repeated_names list
            deckList.extend([name] * amount)

    return champ, wins/12, deckList, hasData

In [16]:
import csv
import json

In [17]:

# Path to the JSON file
#file_path = 'my_dict.json'
file_path = 'drive/MyDrive/deck_pred/my_dict (1).json'
# Load JSON file and convert it to a dictionary
with open(file_path, 'r') as file:
    encodedDeck = json.load(file)


In [18]:


def encodeDeck(deck):
    stats_list = []
    desc_list = []
    for name in deck:
        if name in encodedDeck:
            data = encodedDeck[name]
            stats = data[0]
            desc = data[1]
            stats_list.append(stats)
            desc_list.append(desc)


    return [stats_list, desc_list]

In [19]:
classes = ['Death Knight','Demon Hunter','Druid','Hunter','Mage','Paladin','Preist','Rogue','Shaman','Warlock','Warrior']

# Initialize an empty dictionary to hold the encoding
encoded_classes = {}

# Iterate over each class name
for name in classes:
    # Check if the name is already in the dictionary
    if name not in encoded_classes:
        # If not, add it with a unique integer
        encoded_classes[name] = len(encoded_classes) + 1



def encodeClass(classData):
    empty_list1 = [0] * len(encoded_classes)
    empty_list2 = [0] * len(encoded_classes)
    separated_names = classData.split(' + ')
    empty_list1[encoded_classes.get(separated_names[0])-1]+=1
    empty_list2[encoded_classes.get(separated_names[1])-1]+=1
    return empty_list1+empty_list2


In [20]:
def scrapeArenaData(linksList):
    # Initialize an empty list to store the outputs
    outputs = []

        # Iterate through linksList
    for link in linksList:
        # Call getStatsFromRun for each link and append the result to outputs

        output = getStatsFromRun(link)
        outputs.append(output)

    return outputs

In [21]:
def getTrainingData(arenaData):
    winRateList = []
    DeckList = []
    classList = []
    descList = []

    for run in arenaData:
        print(run[3])
        if run[3] == 1:
            classEncoded = encodeClass(run[0])

            winRate = run[1]
            deckEncoded = encodeDeck(run[2])

            classList.append(classEncoded)
            DeckList.append(deckEncoded[0])
            descList.append(deckEncoded[1])
            winRateList.append(winRate)


    return [classList,DeckList,descList], winRateList


In [22]:
session = login("mcbrideslade@gmail.com", "U$!*3YFvJS2@Dvd4")
arenaUrl = 'https://www.heartharena.com/my-arenas'
linksList = getRuns(arenaUrl,session)

Login successful!


In [23]:
arenaData = scrapeArenaData(linksList)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [24]:
trainingdata, winratedata = getTrainingData(arenaData)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [25]:
inputdata1 = trainingdata[0]
inputdata2 = trainingdata[1]
inputdata3 = trainingdata[2]

inputdatanp1 = np.array(inputdata1, dtype=np.float32)
inputdatanp2 = np.array(inputdata2, dtype=np.float32)
inputdatanp3 = np.array(inputdata3, dtype=np.float32)

winratedatanp = np.array(winratedata, dtype=np.float32)
print(inputdatanp1.shape)
print(inputdatanp2.shape)
print(inputdatanp3.shape)

(25, 22)
(25, 30, 5)
(25, 30, 30, 300)


In [26]:
# Define the custom DeepSetLayer with Dropout
class DeepSetLayer(layers.Layer):
    def __init__(self, output_dim, phi_units=[12, 12], rho_units=[12, 12], dropout_rate=0.2):
        super(DeepSetLayer, self).__init__()
        self.phi_layers = []
        for units in phi_units:
            self.phi_layers.append(layers.Dense(units, activation='gelu'))
            self.phi_layers.append(layers.Dropout(dropout_rate))

        self.rho_layers = []
        for units in rho_units:
            self.rho_layers.append(layers.Dense(units, activation='gelu'))
            self.rho_layers.append(layers.Dropout(dropout_rate))

        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs, training=False):
        # Apply the transformation function Φ to each element
        x = inputs
        for layer in self.phi_layers:
            if isinstance(layer, layers.Dropout):
                x = layer(x, training=training)
            else:
                x = layer(x)

        # Aggregate the transformed elements (averaging)
        #x = tf.reduce_mean(x, axis=1)
        x = tf.reduce_max(x, axis=1)

        # Apply the aggregation function ρ
        for layer in self.rho_layers:
            if isinstance(layer, layers.Dropout):
                x = layer(x, training=training)
            else:
                x = layer(x)

        # Output layer
        output = self.output_layer(x)
        return output

In [27]:
class MaskingLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MaskingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(MaskingLayer, self).build(input_shape)

    def call(self, inputs):
        stats_input, words_input = inputs

        # Generate a binary mask
        batch_size = tf.shape(stats_input)[0]
        num_cards = tf.shape(stats_input)[1]

        # Randomly choose cards to mask
        mask = tf.random.uniform(shape=(batch_size, num_cards), minval=0, maxval=2, dtype=tf.int32)

        # Ensure the same mask is used for both inputs
        mask = tf.cast(mask, dtype=tf.float32)

        # Apply mask to both inputs
        masked_stats = stats_input * mask[:, :, tf.newaxis]
        masked_words = words_input * mask[:, :, tf.newaxis, tf.newaxis]

        return masked_stats, masked_words




In [ ]:
class CardProcessingLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CardProcessingLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.lstm_1 = tf.keras.layers.LSTM(20,return_sequences=False)


    def call(self, inputs, training=False):

        lstm_1 = tf.map_fn(lambda x: self.lstm_1(x), inputs, dtype=tf.float32)

        return lstm_1



In [28]:
 pooled_outputs = tf.reduce_mean(inputs, axis=1)

NameError: name 'inputs' is not defined

In [85]:
class RandomCardSampler(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(RandomCardSampler, self).__init__(**kwargs)

    def call(self, inputs):
        # Determine the maximum number of samples allowed
        num_cards = tf.shape(inputs)[0]
        num_samples = tf.random.uniform(shape=(), minval=1, maxval=num_cards + 1, dtype=tf.int32)

        # Generate unique indices for sampling
        unique_indices = tf.range(num_samples)

        # Randomly shuffle the unique indices
        shuffled_indices = tf.random.shuffle(unique_indices)

        # Gather the sampled cards using the shuffled indices
        sampled_cards = tf.gather(inputs, shuffled_indices[:num_samples])

        output = tf.reduce_mean(sampled_cards,axis=0)

        return output

In [92]:
random_floats = tf.random.normal(shape=(30, 5))
RandomCardSampler()(random_floats)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.50313574,  0.24969801, -0.05865746, -0.35291946,  0.38410187],
      dtype=float32)>

In [35]:
print(tf.random.uniform(shape=(30,), minval=1, maxval = 31, dtype=tf.int32))

tf.Tensor(
[ 3 11 20 26 25 19  5 16 27 25  9  2  3  2 22  6  5 16  8  6  5 11 30  3
 17  4 12 16 23 10], shape=(30,), dtype=int32)


In [ ]:
# Define the inputs
Class = tf.keras.Input(shape=(22,), name='Class')
Deck_stats = tf.keras.Input(shape=(None, 5), name='Deck_stats')
Card_desc = tf.keras.Input(shape=(None, 30, 300), name='Card_desc')

Deck_stats_rem, Card_desc_rem = RandomCardSelector()([Deck_stats, Card_desc])



# Apply 1D Convolution
conv1d = CardProcessingLayer()(Card_desc_rem)




# Create the network for input_1
x1 = Dropout(.5)(Class)

x2 = DeepSetLayer(output_dim=20)(Deck_stats_rem)

x2 = BatchNormalization()(x2)

x3 = DeepSetLayer(output_dim=20)(conv1d)

x3 = BatchNormalization()(x3)

# Concatenate the outputs of the two networks
concatenated = layers.concatenate([x1, x2, x3])

x4 = layers.Dense(50, activation='gelu')(concatenated)

x4 = Dropout(.2)(x4)

# Add an output layer
output = layers.Dense(1)(x4)

# Define the model
model = Model(inputs=[Class, Deck_stats, Card_desc], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.summary()

In [ ]:
# Train the model
model.fit([inputdatanp1,inputdatanp2, inputdatanp3], winratedatanp, epochs=500, batch_size=32)

In [ ]:
prediction = model.predict([inputdatanp1,inputdatanp2, inputdatanp3])
flattened_list = [item for sublist in prediction for item in sublist]

In [ ]:
list1 = winratedatanp
list2 = flattened_list

In [ ]:
# Step 4: Combine arrays
combined_array = np.vstack([list1, list2])
# Extract the first row (keys)
first_row = combined_array[0]

# Sort the combined_array based on the first row
sorted_combined_array = np.array(sorted(zip(first_row, *combined_array[1:]), key=lambda x: x[0]))

In [ ]:
# Separate the two columns into lists
column_1 = sorted_combined_array[:, 0].tolist()
column_2 = sorted_combined_array[:, 1].tolist()

In [ ]:
# Plotting
plt.plot(list1, label='List 1', color='blue')
plt.plot(list2, label='List 2', color='red')

# Customization
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Two Lists Overlay')
plt.legend()

# Display the plot
plt.show()